### Imports

In [3]:
import blockcypher
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import time

# All the dates in the blockchain are UTC
UTC=pytz.UTC

## Nota, si no funciona pip: python -m ensurepip

In [4]:
# for the debate of static class with static methods vs library: in a jupyter notebook, to create a static class is more coherent. 
# All the methods under the same class and in the same cell. Easier to translate to an Script if it is needed.
class BlckChainAnalyser:
    @staticmethod
    def calculate_balance_from_txr(trxs: list)-> int:
        """Calculate the balance from a list of txr. The function doesn't check if the transactions are in the same block

        Args:
            trxs (list): List of transactions in the format returned by blockcypher API

        Returns:
            int: Sum of all the values in the list of transactions
        """        
        assert trxs is not None

        #starting_balance = trxs[-1]["ref_balance"]-trxs[-1]["value"]
        acc_balance = 0
        for trx in trxs:
            acc_balance = np.sign(trx["tx_output_n"])*trx["value"]+acc_balance
        return acc_balance
    
    @staticmethod
    def trxs_filter_by_date(trxs: list, init_date: datetime, end_date: datetime)-> list:
        """Filter the list of BTC transactions passed by argument and obtained using blockcypher API to be between init_date end_date

        Args:
            trxs (list): List of transactions in the format returned by blockcypher API
            init_date (datetime): Initial datetime of the range. Needs to be UTC
            end_date (datetime): End datetime of the range. Needs to be UTC

        Returns:
            list: A sublist of the initial list containing transactions above or equal to init_date and strictily bellow to end_date. An empty list if there are no
            transactions in that range of dates
        """
        assert init_date.tzname() == "UTC"
        assert end_date.tzname() == "UTC"
        res = [trx for trx in trxs if init_date <= trx["confirmed"] < end_date]
        #list_filtered = filter((lambda x: x["confirmed"]>=init_date and x["confirmed"]<end_date),trxs)
        return res
    
    @staticmethod
    def trxs_filter_by_address(trxs: list, wallet_address: str)-> list:
        """Filters the list of BTC transactions by address

        Args:
            trxs (list): Lists of transactions in the format returned by blockcypher
            wallet_address (str): Wallet address

        Raises:
            err: [description]

        Returns:
            list: List of transactions filtered by wallet
        """
        assert len(trxs) > 0
        res = [trx for trx in trxs if trx['address'] == wallet_address]
        return res
    
    @staticmethod
    def get_details_addresses_low_profile(list_addresses: list, wait: float=0.5)-> list:
        """Make n calls to get_address_details being n the len of list_addresses. It will wait [wait] seconds between each call, to avoid ERROR 429 from the server 

        Args:
            list_addresses (list): List containing all the addresses to be query
            wait (float): Seconds between each call to avoid error from the server
        Returns:
            list: list of dict cotaing: {"address": "<address>", "details": {<details returned from the server>}}
        """        
        details_addresses = []
        for addr in list_addresses:
            try:
                details_addresses_temp = blockcypher.get_address_details(addr)
            except Exception as err:
                print(len(details_addresses))
                raise err
                
            details_addresses.append({"address": addr, "details":details_addresses_temp})
            time.sleep(wait)
        return details_addresses

---
### Reading the data of the 500 biggest BTC addresses


In [5]:
# from here: https://bitinfocharts.com/top-100-richest-bitcoin-addresses.html
big_btc_addr = pd.read_csv("../data/500_biggest_btc_add.csv")
# Spliting the address and the alias
result = big_btc_addr['address'].str.split(n=2,expand=True)
big_btc_addr.drop(columns=["address"], inplace=True, errors="ignore")
big_btc_addr['alias'] = result[2]
big_btc_addr['address'] = result[0]
# Droping all the rows in the DF that are NaN
big_btc_addr.dropna(subset=["address"], inplace=True)
list_of_big_addresses = big_btc_addr.address.to_list()
# We are expecting 500 addresses
assert(len(list_of_big_addresses)==500)

In [6]:
addresses_details = BlckChainAnalyser.get_details_addresses_low_profile(list_of_big_addresses[0:10])

In [7]:
print(len(addresses_details))
print(len(addresses_details[5]["details"]["txrefs"]))

10
200


In [8]:
binance_cold_address = "34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo"
binance_cold_trxs = BlckChainAnalyser.trxs_filter_by_address(addresses_details, binance_cold_address)[0]

##### Convert to pandas a transaction list. Binance cold list of transactions 

In [9]:
binance_cold_trxs['unconfirmed_txrefs'] = 0

In [10]:
binance_cold_trxs['details']['unconfirmed_txrefs'] = 0

In [11]:
binance_cold_df = pd.DataFrame.from_dict(binance_cold_trxs['details'])

In [12]:
binance_cold_df_exploded = binance_cold_df.explode('txrefs')

In [13]:
binance_cold_df_exploded

,address,total_received,total_sent,balance,unconfirmed_balance,final_balance,n_tx,unconfirmed_n_tx,final_n_tx,txrefs,hasMore,tx_url,unconfirmed_txrefs
0,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389806338602,84130083091124,25259723247478,0,25259723247478,745,0,745,tx_hash,True,https://api.blockcypher.com/v1/btc/main/txs/,0
0,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389806338602,84130083091124,25259723247478,0,25259723247478,745,0,745,block_height,True,https://api.blockcypher.com/v1/btc/main/txs/,0
0,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389806338602,84130083091124,25259723247478,0,25259723247478,745,0,745,tx_input_n,True,https://api.blockcypher.com/v1/btc/main/txs/,0
0,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389806338602,84130083091124,25259723247478,0,25259723247478,745,0,745,tx_output_n,True,https://api.blockcypher.com/v1/btc/main/txs/,0
0,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389806338602,84130083091124,25259723247478,0,25259723247478,745,0,745,value,True,https://api.blockcypher.com/v1/btc/main/txs/,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389806338602,84130083091124,25259723247478,0,25259723247478,745,0,745,ref_balance,True,https://api.blockcypher.com/v1/btc/main/txs/,0
199,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389806338602,84130083091124,25259723247478,0,25259723247478,745,0,745,spent,True,https://api.blockcypher.com/v1/btc/main/txs/,0
199,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389806338602,84130083091124,25259723247478,0,25259723247478,745,0,745,confirmations,True,https://api.blockcypher.com/v1/btc/main/txs/,0
199,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389806338602,84130083091124,25259723247478,0,25259723247478,745,0,745,confirmed,True,https://api.blockcypher.com/v1/btc/main/txs/,0


In [14]:
binance_cold = blockcypher.get_address_details("34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo")

In [15]:
binance_cold 

{'address': '34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo',
 'total_received': 109389806338602,
 'total_sent': 84130083091124,
 'balance': 25259723247478,
 'unconfirmed_balance': 0,
 'final_balance': 25259723247478,
 'n_tx': 745,
 'unconfirmed_n_tx': 0,
 'final_n_tx': 745,
 'txrefs': [{'tx_hash': '2816e7a5ce6b21f48fa8a0f6ffe4d6f5e811cf7795cae224614ec50dbc09528b',
   'block_height': 732302,
   'tx_input_n': -1,
   'tx_output_n': 1,
   'value': 618,
   'ref_balance': 25259723247478,
   'spent': False,
   'confirmations': 110,
   'confirmed': datetime.datetime(2022, 4, 17, 16, 35, 24, tzinfo=tzutc()),
   'double_spend': False},
  {'tx_hash': 'd429d02629d62e796508c5cfcabd3a7cb4d20c6ce250c5bea01ea1e4c1d9e65c',
   'block_height': 731242,
   'tx_input_n': -1,
   'tx_output_n': 0,
   'value': 4684,
   'ref_balance': 25259723246860,
   'spent': False,
   'confirmations': 1170,
   'confirmed': datetime.datetime(2022, 4, 10, 8, 26, 56, tzinfo=tzutc()),
   'double_spend': False},
  {'tx_hash': '9c4eb173b679

In [16]:
binance_cold_trxs

{'address': '34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo',
 'details': {'address': '34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo',
  'total_received': 109389806338602,
  'total_sent': 84130083091124,
  'balance': 25259723247478,
  'unconfirmed_balance': 0,
  'final_balance': 25259723247478,
  'n_tx': 745,
  'unconfirmed_n_tx': 0,
  'final_n_tx': 745,
  'txrefs': [{'tx_hash': '2816e7a5ce6b21f48fa8a0f6ffe4d6f5e811cf7795cae224614ec50dbc09528b',
    'block_height': 732302,
    'tx_input_n': -1,
    'tx_output_n': 1,
    'value': 618,
    'ref_balance': 25259723247478,
    'spent': False,
    'confirmations': 110,
    'confirmed': datetime.datetime(2022, 4, 17, 16, 35, 24, tzinfo=tzutc()),
    'double_spend': False},
   {'tx_hash': 'd429d02629d62e796508c5cfcabd3a7cb4d20c6ce250c5bea01ea1e4c1d9e65c',
    'block_height': 731242,
    'tx_input_n': -1,
    'tx_output_n': 0,
    'value': 4684,
    'ref_balance': 25259723246860,
    'spent': False,
    'confirmations': 1170,
    'confirmed': datetime.datetime(2022, 

In [18]:
list_filtered = BlckChainAnalyser.trxs_filter_by_date(binance_cold_trxs['details']["txrefs"], datetime(2021,12,7, tzinfo=UTC), datetime(2021,12,8, tzinfo=UTC))
BlckChainAnalyser.calculate_balance_from_txr(list_filtered)

-2264653016196

In [19]:
addresses_details

[{'address': '34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo',
  'details': {'address': '34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo',
   'total_received': 109389806338602,
   'total_sent': 84130083091124,
   'balance': 25259723247478,
   'unconfirmed_balance': 0,
   'final_balance': 25259723247478,
   'n_tx': 745,
   'unconfirmed_n_tx': 0,
   'final_n_tx': 745,
   'txrefs': [{'tx_hash': '2816e7a5ce6b21f48fa8a0f6ffe4d6f5e811cf7795cae224614ec50dbc09528b',
     'block_height': 732302,
     'tx_input_n': -1,
     'tx_output_n': 1,
     'value': 618,
     'ref_balance': 25259723247478,
     'spent': False,
     'confirmations': 110,
     'confirmed': datetime.datetime(2022, 4, 17, 16, 35, 24, tzinfo=tzutc()),
     'double_spend': False},
    {'tx_hash': 'd429d02629d62e796508c5cfcabd3a7cb4d20c6ce250c5bea01ea1e4c1d9e65c',
     'block_height': 731242,
     'tx_input_n': -1,
     'tx_output_n': 0,
     'value': 4684,
     'ref_balance': 25259723246860,
     'spent': False,
     'confirmations': 1170,
     'confirm

## Checking the delta of these addresses in the last 24 hours

In [20]:
blockcypher.get_address_details("1NDyJtNTjmwk5xPNhjgAMu4HDHigtobu1s")

{'address': '1NDyJtNTjmwk5xPNhjgAMu4HDHigtobu1s',
 'total_received': 1558453578217144,
 'total_sent': 1558453567666568,
 'balance': 10550576,
 'unconfirmed_balance': 0,
 'final_balance': 10550576,
 'n_tx': 1191620,
 'unconfirmed_n_tx': 0,
 'final_n_tx': 1191620,
 'txrefs': [{'tx_hash': '0639a0b06a54e5bf6dd7d60ec30d3e38b0914c80e14be3b8769ca2766fc53d63',
   'block_height': 730985,
   'tx_input_n': -1,
   'tx_output_n': 7,
   'value': 387000,
   'ref_balance': 10550576,
   'spent': False,
   'confirmations': 1427,
   'confirmed': datetime.datetime(2022, 4, 8, 15, 9, 10, tzinfo=tzutc()),
   'double_spend': False},
  {'tx_hash': 'fa29cb89e788a4308410f96e5ef1ace7a50015d06ad0cde169d60ce25c91df1e',
   'block_height': 730728,
   'tx_input_n': -1,
   'tx_output_n': 0,
   'value': 297312,
   'ref_balance': 10163576,
   'spent': False,
   'confirmations': 1684,
   'confirmed': datetime.datetime(2022, 4, 6, 18, 24, 8, tzinfo=tzutc()),
   'double_spend': False},
  {'tx_hash': '96cf8583ff818682c22055

 UNIT TEST
-----

###### from here: https://stackoverflow.com/questions/40172281/unit-tests-for-functions-in-a-jupyter-notebook

In [21]:
import unittest

class TestBlckChainAnalyser(unittest.TestCase):
    
    def test__balance_calc(self):
        """ Testing a real world case, using the binance_cold wallet and the transaction done at 2021-12-07 06:06:09 
            here: https://bitinfocharts.com/bitcoin/block/712989/34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo?__cf_chl_tk=it9k.tfmmjZpvgQ87dWZTXqRvVqvwNg_lGmqJF.iu4Y-1640963834-0-gaNycGzNCeU
        """        
        binance_cold_wallet = blockcypher.get_address_details("34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo")
        trxs_filtered = BlckChainAnalyser.trxs_filter_by_date(binance_cold_wallet["txrefs"], datetime(2021, 12, 7, tzinfo=UTC), datetime(2021, 12, 8, tzinfo=UTC))
        self.assertEqual(BlckChainAnalyser.calculate_balance_from_txr(trxs_filtered), -2264653016196)

unittest.main(argv=[''], verbosity=2, exit=False)

test__balance_calc (__main__.TestBlckChainAnalyser)
Testing a real world case, using the binance_cold wallet and the transaction done at 2021-12-07 06:06:09 ... ok

----------------------------------------------------------------------
Ran 1 test in 0.282s

OK
